In [5]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import cv2
import os
from keras.models import Sequential
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense

img_width = 128
img_height = 96

In [6]:
def get_number(num):
    if num < 10:
        return "0" + str(num)
    return str(num)

In [16]:
def video_to_frames(video_path, output_dir, num_frames=60):
    vidcap = cv2.VideoCapture(video_path)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))  # get the total number of frames
    start_frame = max(0, total_frames - num_frames)  # calculate the starting frame for the last n frames

    success, image = vidcap.read()
    count = 0
    while success:
        if count >= start_frame:  # only save frames from the last n frames
            height, width, _ = image.shape  # get the dimensions of the frame
            right_half = image[:, width//2:]  # select the right half of the frame
            cv2.imwrite(output_dir + "/frame" + get_number(count - start_frame) + ".jpg", right_half)  # save frame as JPEG file
        success, image = vidcap.read()
        count += 1

In [17]:
directory = "C:/Users/M/OneDrive - softromic/Documents/TreeHacks Fall Detection/falls"
dest = "C:/Users/M/OneDrive - softromic/Documents/TreeHacks Fall Detection/video frames/vidframes - "
for i in range(30):
    video_to_frames(directory + "/fall-" + str(i+1) + "-cam0.mp4", dest + str(i+1))

In [22]:
directory = "C:/Users/M/OneDrive - softromic/Documents/TreeHacks Fall Detection/nonfalls"
dest = "C:/Users/M/OneDrive - softromic/Documents/TreeHacks Fall Detection/nonfall video frames/vidframes - "
for i in range(40):
    video_to_frames(directory + "/adl-" + str(i+1) + "-cam0.mp4", dest + str(i+1))

In [23]:
def load_frames_from_folder(folder_path):
    images = []
    for filename in sorted(os.listdir(folder_path)):  # assuming all frames are named in ascending order
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (img_width, img_height))
            img = img / 255
            images.append(img)
    images = np.array(images)
    return images

In [24]:
def compile_frames(dir, num_frames):

    frames = []

    for video_folder in os.listdir(dir):
        video_frames = load_frames_from_folder(os.path.join(dir, video_folder))
        video_frames = np.array(video_frames)
        
        # Ensure all videos have the same number of frames
        if len(video_frames) < num_frames:
            # If the video has fewer than num_frames frames, pad it with zeros
            padding = np.zeros((num_frames - len(video_frames),) + video_frames.shape[1:])
            video_frames = np.concatenate([video_frames, padding])
        elif len(video_frames) > num_frames:
            # If the video has more than num_frames frames, truncate it
            video_frames = video_frames[:num_frames]
        frames.append(video_frames)
    
    return frames

In [26]:
num_frames = 60

fall_frame_dir = "C:/Users/M/OneDrive - softromic/Documents/TreeHacks Fall Detection/video frames"
fall_videos_frames = compile_frames(fall_frame_dir, num_frames)

nonfall_frame_dir = "C:/Users/M/OneDrive - softromic/Documents/TreeHacks Fall Detection/nonfall video frames"
nonfall_videos_frames = compile_frames(nonfall_frame_dir, num_frames)

all_videos_frames = fall_videos_frames
for i in range(40):
    all_videos_frames.append(nonfall_videos_frames[i])

all_videos_frames = np.array(all_videos_frames)

In [27]:
all_videos_labels = []
for i in range(30):
    all_videos_labels.append(1)
for i in range(40):
    all_videos_labels.append(0)

all_videos_labels =  np.array(all_videos_labels)

In [28]:
frame_shape = (img_height, img_width, 3)  # Height, Width, Channels

# Define the number of frames in each sequence.
sequence_length = 60

model = Sequential([
    # TimeDistributed wrapper allows applying the same layers individually to each time step.
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(sequence_length,) + frame_shape),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    # Flatten the output of the conv layers so it can be fed into the LSTM layer
    TimeDistributed(Flatten()),
    # LSTM layer for analyzing the time series data
    LSTM(64),
    # Dense layers for classification
    Dense(64, activation='relu'),
    # Output layer for binary classification. Use 'sigmoid' activation for binary classification.
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 60, 96, 128, 32)   896       
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 60, 48, 64, 32)    0         
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 60, 48, 64, 64)    18496     
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 60, 24, 32, 64)    0         
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 60, 24, 32, 128) 

In [29]:
indices = np.arange(all_videos_frames.shape[0])
np.random.shuffle(indices)

all_videos_frames = all_videos_frames[indices]
all_videos_labels = all_videos_labels[indices]

model.fit(all_videos_frames, all_videos_labels, epochs=10, validation_split=0.2)

Epoch 1/10


2/2 [==============================] - 102s 42s/step - loss: 0.9720 - accuracy: 0.5179 - val_loss: 0.7044 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 76s 37s/step - loss: 0.7060 - accuracy: 0.6071 - val_loss: 0.7269 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 72s 29s/step - loss: 0.6882 - accuracy: 0.5893 - val_loss: 0.7382 - val_accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 73s 36s/step - loss: 0.6503 - accuracy: 0.5893 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 60s 28s/step - loss: 0.6094 - accuracy: 0.5893 - val_loss: 0.6479 - val_accuracy: 0.5000
Epoch 6/10
2/2 [==============================] - 80s 35s/step - loss: 0.6687 - accuracy: 0.5893 - val_loss: 0.7108 - val_accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 84s 34s/step - loss: 0.5831 - accuracy: 0.5893 - val_loss: 0.5806 - val_accuracy: 0.5000
Epoch 8/10
2/2 [========

In [30]:
model.save("C:/Users/M/OneDrive - softromic/Documents/TreeHacks Fall Detection/models/model.keras")